# Utilities 

In [24]:
import pandas as pd

### Data Clean
1. Parse Excel
2. Clean Sales tab (upsample, fix date format)

In [25]:
excel_file = pd.ExcelFile('raw_data.xlsx')

sheet_names = excel_file.sheet_names

data_frames = {}

for sheet_name in sheet_names:
    data_frames[sheet_name] = excel_file.parse(sheet_name)

competitor_sales_df = data_frames['competitor_sales']

### Clean Sales + Events + Holidays files: 
1. date_time format
2. fill na(0) for missing product sale data
3. index per date
4. sort old to new
5. clean annomalies (negative values in sale, outliers)

In [26]:
sales_df = data_frames['sales']
sales_df = data_frames['sales'].rename(columns = {'sales_month' : 'date'})
sales_df['sales'].abs()
sales_df['date'] = pd.to_datetime(sales_df['date'], format='%Y-%m-%dT%H:%M:%S.%f%z').dt.strftime('%d-%m-%Y')
sales_df = sales_df.set_index(['date', 'category']).unstack(fill_value=0).stack().reset_index().sort_values('date')

In [27]:
for index, value in sales_df['sales'].items(): 
    
    average = sales_df['sales'].mean()
    
    if value > average * 100:
        sales_df.at[index, 'sales'] = 0
        
    elif value < 0:
        sales_df.at[index, 'sales'] = abs(value)
sales_df.head(20)

,date,category,sales
0,01-01-2020,b,168.957105
1,01-01-2020,c,605.214724
2,01-01-2020,d,2534.086679
3,01-01-2020,e,699.861172
4,01-01-2020,f,206.322257
5,01-01-2020,g,0.000000
6,01-01-2021,b,197.148008
7,01-01-2021,c,636.498179
8,01-01-2021,d,2617.005001
9,01-01-2021,e,832.801780


In [28]:
holidays_df = data_frames['holidays']
holidays_df['date'] = pd.to_datetime(holidays_df['date'], format='%d-%m-%Y')
holidays_df = holidays_df.sort_values('date', ascending=True).set_index('date')

holidays_df.head()

,holiday
date,
2020-03-09,Purim - Eve
2020-03-10,Purim
2020-03-11,Shushan Purim
2020-04-08,Passover I - Eve
2020-04-09,Passover I


In [29]:
events_df = data_frames['events']
events_df['date'] = pd.to_datetime(events_df['date'], format='%d-%m-%Y')
events_df = events_df.sort_values('date', ascending=True).set_index('date')

events_df.head()

,event_name
date,
2020-04-01,MediaCampaign
2020-04-02,MediaCampaign
2020-04-03,MediaCampaign
2020-04-04,MediaCampaign
2020-04-05,MediaCampaign


save to pkl all 4 files

In [30]:
sales_df.to_pickle('sales.pkl')
events_df.to_pickle('events.pkl')
holidays_df.to_pickle('holidays.pkl')
competitor_sales_df.to_pickle('competitor_sales.pkl')